In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE99725"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE99725"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE99725.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE99725.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE99725.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import numpy as np
import json
from typing import Callable, Optional, Dict, Any

# Step 1: Gene Expression Data Availability
# Based on the background info, this seems to be a study of whole-genome expression profiling
# from peripheral blood, so gene expression data should be available
is_gene_available = True

# Step 2: Clinical Variable Analysis
# 2.1 Data Availability
# Analyzing the sample characteristics dictionary:
# Key 0: Contains patient identifiers
# Key 1: Shows time points (M0, M6) - Before and after bariatric surgery
# Key 2: Shows MADRS (Montgomery-Asberg Depression Rating Scale) values - A and B likely indicate depression status
# Key 3: Shows tissue type (Venous blood)

# For trait (Obesity):
# The study is about "comorbid depression in obese patients" and all patients are described as "massively obese"
# However, the dataset doesn't seem to have BMI or obesity metrics, so all patients likely have the same obesity status
# This makes obesity a constant feature, not suitable for our analysis

# For depression (MADRS):
# Key 2 contains MADRS values which indicate depression status (A/B likely mean depressed/non-depressed)
# This can be our trait of interest

trait_row = 2  # MADRS values in key 2
age_row = None  # Age not available in the sample characteristics
gender_row = None  # Gender not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert MADRS values to binary depression status (0=non-depressed, 1=depressed)"""
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip()
    if value == 'A':
        return 1  # Depressed (assuming A means high MADRS score)
    elif value == 'B':
        return 0  # Non-depressed (assuming B means low MADRS score)
    else:
        return None

def convert_age(value):
    """Placeholder function as age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function as gender data is not available"""
    return None

# Step 3: Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame that has the same structure as what the geo_select_clinical_features function expects
    # Each column represents a characteristic type, and contains all possible values for that characteristic
    sample_chars = {
        0: ['patient: CB291013', 'patient: TP100414', 'patient: JDF280314', 'patient: JA021214', 
            'patient: DC160914', 'patient: GMD170315', 'patient: MP220714', 'patient: SM260215', 
            'patient: MC261113', 'patient: SB091214', 'patient: CN220714', 'patient: AE170614', 
            'patient: AG121114', 'patient: SS150414', 'patient: TDC270115', 'patient: VF200115', 
            'patient: KP261113', 'patient: AC030215', 'patient: SM070415', 'patient: JMV220115', 
            'patient: NC130214', 'patient: SB221013', 'patient: MA021214', 'patient: DD101214', 
            'patient: LB141114', 'patient: CPP281113', 'patient: NR180314', 'patient: PP120315', 
            'patient: BB080414', 'patient: PM120914'],
        1: ['time: M0', 'time: M6'],
        2: ['MADRS: A', 'MADRS: B'],
        3: ['tissue: Venous blood']
    }
    
    # Convert the dictionary to a format expected by geo_select_clinical_features
    # We need a DataFrame where each row corresponds to a feature type, not samples
    clinical_data = pd.DataFrame()
    for row_id, values in sample_chars.items():
        clinical_data[row_id] = values if row_id < len(clinical_data) else values + [None] * (len(clinical_data) - len(values))
    
    # Fill any missing values with None
    clinical_data = clinical_data.fillna(None)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,  # Using the predefined trait variable
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    print(preview_df(selected_clinical_df))
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# 1. Import necessary libraries
import pandas as pd
import numpy as np
import os
import glob
import re

# 2. First, let's explore what files are available in the directory
print(f"Exploring directory: {in_cohort_dir}")
available_files = os.listdir(in_cohort_dir)
print(f"Available files: {available_files}")

# Load clinical/phenotype data if available
clinical_data = None
sample_characteristic_files = [f for f in available_files if 'characteristics' in f.lower() or 'phenotype' in f.lower() or 'clinical' in f.lower()]

if sample_characteristic_files:
    clinical_data_path = os.path.join(in_cohort_dir, sample_characteristic_files[0])
    print(f"Loading clinical data from: {clinical_data_path}")
    clinical_data = pd.read_csv(clinical_data_path, sep=None, engine='python')
    print("Clinical data shape:", clinical_data.shape)
    print("Sample from clinical data:")
    print(clinical_data.head())

# Load matrix data to check if gene expression data is available
matrix_files = [f for f in available_files if 'matrix' in f.lower() or '.txt' in f.lower() or '.tsv' in f.lower() or '.csv' in f.lower()]
is_gene_available = False

for file in matrix_files:
    if 'matrix' in file.lower() or 'expression' in file.lower() or 'gene' in file.lower():
        # This is likely gene expression data
        is_gene_available = True
        break

# Load background information if available
background_info = ""
background_files = [f for f in available_files if 'background' in f.lower() or 'readme' in f.lower() or 'info' in f.lower()]
if background_files:
    background_path = os.path.join(in_cohort_dir, background_files[0])
    print(f"Loading background information from: {background_path}")
    try:
        with open(background_path, 'r') as f:
            background_info = f.read()
        print("Background information preview:")
        print(background_info[:500] + "..." if len(background_info) > 500 else background_info)
    except Exception as e:
        print(f"Error reading background file: {e}")

# If clinical data is available, analyze it to identify trait, age, and gender rows
trait_row = None
age_row = None
gender_row = None
is_trait_available = False

if clinical_data is not None:
    print("\nAnalyzing clinical data to identify variables...")
    # Check unique values in each row to identify relevant variables
    for i in range(len(clinical_data)):
        try:
            row_description = str(clinical_data.iloc[i, 0]).lower() if clinical_data.shape[1] > 0 else ""
            row_values = clinical_data.iloc[i, 1:].astype(str) if clinical_data.shape[1] > 1 else pd.Series([])
            unique_vals = row_values.unique()
            
            if len(unique_vals) > 1:  # Only consider rows with more than one unique value
                print(f"Row {i}: {row_description}")
                print(f"  Unique values: {unique_vals[:5]}{'...' if len(unique_vals) > 5 else ''}")
                
                # Check for obesity-related trait (BMI, weight, obesity status)
                if any(term in row_description for term in ['bmi', 'body mass', 'weight', 'obes', 'adipos']):
                    trait_row = i
                    print(f"  Identified as trait row (Obesity)")
                
                # Check for age
                elif any(term in row_description for term in ['age', 'year']):
                    age_row = i
                    print(f"  Identified as age row")
                
                # Check for gender/sex
                elif any(term in row_description for term in ['gender', 'sex', 'male', 'female']):
                    gender_row = i
                    print(f"  Identified as gender row")
        except Exception as e:
            print(f"Error analyzing row {i}: {e}")

    # Check if trait data is available
    is_trait_available = trait_row is not None

# Define conversion functions for each variable
def convert_trait(value):
    """Convert obesity-related trait values to numeric."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to numeric (for BMI or weight values)
    try:
        return float(value)
    except (ValueError, TypeError):
        # Handle categorical values
        value_lower = str(value).lower()
        if 'obese' in value_lower or 'obesity' in value_lower:
            return 1
        elif 'normal' in value_lower or 'lean' in value_lower or 'control' in value_lower:
            return 0
        else:
            return None

def convert_age(value):
    """Convert age values to numeric."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    try:
        # Remove non-numeric characters but keep decimals
        value_str = str(value).lower()
        if 'year' in value_str:
            value_str = value_str.split('year')[0].strip()
        
        # Extract numbers using regex
        matches = re.findall(r'\d+\.?\d*', value_str)
        if matches:
            return float(matches[0])
        return None
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value_lower = str(value).lower()
    if any(term in value_lower for term in ['female', 'f', 'woman', 'women']):
        return 0
    elif any(term in value_lower for term in ['male', 'm', 'man', 'men']):
        return 1
    else:
        return None

# Validate and save initial cohort info
print(f"\nSaving initial cohort info...")
print(f"is_gene_available: {is_gene_available}")
print(f"is_trait_available: {is_trait_available}")
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical Feature Extraction (if trait_row is not None and clinical_data is available)
if trait_row is not None and clinical_data is not None:
    print(f"\nExtracting clinical features...")
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical data
    print("Preview of extracted clinical data:")
    preview_data = preview_df(selected_clinical_df)
    print(preview_data)
    
    # Save the


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Let's load the available data first
print(f"Looking for data in: {in_cohort_dir}")
os.listdir(in_cohort_dir)

# Try to load the sample characteristics data
clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.pkl")
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_pickle(clinical_data_path)
    print("Clinical data loaded")
    print(clinical_data.shape)
    print(clinical_data.head())
else:
    print("Clinical data not found")
    clinical_data = None

# Try to load background information if available
info_path = os.path.join(in_cohort_dir, "info.txt")
if os.path.exists(info_path):
    with open(info_path, 'r') as f:
        info_content = f.read()
    print("Background information loaded")
    print(info_content)
else:
    print("Background information not found")
    info_content = ""

# Display unique values in the clinical data to identify relevant rows
if clinical_data is not None:
    for i in range(len(clinical_data.index)):
        unique_values = clinical_data.iloc[i].unique()
        if len(unique_values) <= 10:  # Only show if the number of unique values is manageable
            print(f"Row {i}: {clinical_data.index[i]}")
            print(f"Unique values: {unique_values}")
            print("---")
        else:
            print(f"Row {i}: {clinical_data.index[i]} (too many unique values to display)")
            print("---")

# 1. Determine if gene expression data is available
# Set this to True if gene expression data is available, otherwise False
is_gene_available = True  # Assuming gene expression data is available unless we find evidence against it

# 2.1 Identify rows containing trait, age, and gender data
trait_row = None
age_row = None
gender_row = None

if clinical_data is not None:
    # Look for obesity-related information
    obesity_keywords = ['obesity', 'bmi', 'weight', 'body mass index', 'body weight']
    for i in range(len(clinical_data.index)):
        row_name = str(clinical_data.index[i]).lower()
        unique_values = clinical_data.iloc[i].unique()
        
        # Check for obesity/BMI related rows
        if any(keyword in row_name for keyword in obesity_keywords) and len(unique_values) > 1:
            trait_row = i
            print(f"Found potential trait row: {i} - {clinical_data.index[i]}")
            print(f"Unique values: {unique_values}")
        
        # Check for age
        if 'age' in row_name and len(unique_values) > 1:
            age_row = i
            print(f"Found potential age row: {i} - {clinical_data.index[i]}")
            print(f"Unique values: {unique_values}")
        
        # Check for gender/sex
        if ('gender' in row_name or 'sex' in row_name) and len(unique_values) > 1:
            gender_row = i
            print(f"Found potential gender row: {i} - {clinical_data.index[i]}")
            print(f"Unique values: {unique_values}")

# 2.2 Define conversion functions for each variable

def convert_trait(value):
    """Convert trait values to binary (0 for non-obese, 1 for obese)"""
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Common mappings for obesity
    if 'obese' in value or 'obesity' in value:
        return 1
    elif 'normal' in value or 'lean' in value or 'control' in value:
        return 0
    
    # Try to extract BMI value
    try:
        # If value contains digits, try to parse as BMI
        import re
        numbers = re.findall(r'\d+\.?\d*', value)
        if numbers:
            bmi = float(numbers[0])
            return 1 if bmi >= 30 else 0  # Standard obesity threshold
    except:
        pass
    
    return None

def convert_age(value):
    """Convert age values to continuous numbers"""
    if pd.isna(value):
        return None
    
    value = str(value)
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age value
    try:
        import re
        numbers = re.findall(r'\d+\.?\d*', value)
        if numbers:
            return float(numbers[0])
    except:
        pass
    
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Common mappings for gender
    if 'female' in value or 'f' == value.strip():
        return 0
    elif 'male' in value or 'm' == value.strip():
        return 1
    
    return None

# 3. Save metadata with initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if trait_row is not None:
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of selected clinical features:")
    preview_result = preview_df(selected_clinical_df)
    print(preview_result)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No clinical features were extracted as trait data is not available.")


### Step 5: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 6: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers in the format 'A_19_P00315452', these appear to be Agilent microarray probe IDs
# rather than standard human gene symbols. Agilent probe IDs typically use this format with 'A_' prefix.
#
# These identifiers need to be mapped to standard gene symbols for proper analysis.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for gene identifiers and gene symbols
# Looking at the gene expression data and gene annotation data:
# - The gene expression data has index with identifiers like 'A_19_P00315452'
# - The gene annotation data has similar identifiers in the 'ID' column
# - The gene symbols are in the 'GENE_SYMBOL' column of the annotation data

# 2. Extract the mapping between probe IDs and gene symbols
prob_col = 'ID'  # Column with probe identifiers 
gene_col = 'GENE_SYMBOL'  # Column with gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Mapping dataframe preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First 10 genes:")
print(gene_data.index[:10])


### Step 9: Data Normalization and Linking

In [ ]:
# 1. First let's verify the file paths and get gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Load and process gene expression data
gene_data = get_genetic_data(matrix_file)

# 3. Get gene annotations and mapping
gene_annotation = get_gene_annotation(soft_file)
prob_col = 'ID'  # Column with probe identifiers 
gene_col = 'GENE_SYMBOL'  # Column with gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 4. Apply gene mapping to convert probe-level data to gene expression data
mapped_gene_data = apply_gene_mapping(gene_data, mapping_df)

# 5. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(mapped_gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 6. Check if clinical data is available (based on previous steps, it's not)
# We'll first get the soft_file and matrix_file again to obtain clinical_data
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

if clinical_data is not None and trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Shape of linked data before missing value handling: {linked_data.shape}")
    
    # Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after missing value handling: {linked_data.shape}")
    
    # Check if the trait and demographic features are severely biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # Validate and save metadata
    note = "Dataset with gene expression data and clinical features."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )
    
    # Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for trait-gene association studies due to quality issues.")
else:
    # Clinical data is not available or trait row not found
    print("Clinical data or trait information not available - cannot proceed with linking or trait analysis")
    
    # Create an empty DataFrame with the same structure that would be expected
    # This allows us to pass it to the validation function
    empty_df = pd.DataFrame(columns=[trait])
    
    # Save metadata indicating trait data is not available
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=empty_df,
        note="Gene expression data available but no clinical data found for trait analysis."
    )
    print("Metadata saved indicating gene data available but trait data unavailable.")


### Step 10: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the function from tools.preprocess
# Use the parameters defined in STEP 2 for the trait, age, and gender rows
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the extracted clinical features to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data before missing value handling: {linked_data.shape}")

# 4. Handle missing values systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after missing value handling: {linked_data.shape}")

# 5. Check if the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
note = "Dataset from NAFLD study with hypertension trait information available."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to quality issues.")